In [2]:
%load_ext Cython

In [3]:
np.zeros(10)

NameError: name 'np' is not defined

In [8]:
%%cython
import numpy as np
cimport numpy as np


a = np.arange(10, dtype=np.float64)

cdef double[:] b = a

print a
print b

b[1] = 22

print a

from libc.string cimport memset
memset(&b[0], 0, len(a)*sizeof(np.float64))

print a

cdef struct Crap:
    double * a
    
cdef Crap crap = Crap(&b[0])

crap.a[4] = 111

print a

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
<MemoryView of 'ndarray' object>
[  0.  22.   2.   3.   4.   5.   6.   7.   8.   9.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[   0.    0.    0.    0.  111.    0.    0.    0.    0.    0.]


In [29]:
b

NameError: name 'b' is not defined

In [21]:
%%cython

cdef int fib(int n):
    if n in (0,1):
        return n
    else:
        return fib(n-1) + fib(n-2)

cdef int * test():
    cdef int *a = [1,2,3]
    print a[1]
    return a
    
def wrapper():
    a = test()
    cdef int *b = [11,21,31]
    
    c = fib(200)
    
    
    print a[1]

In [28]:
import numpy as np
np.zeros(3).dtype

dtype('float64')

In [62]:
import numpy as np
import scipy.sparse as sp


ij = np.array([[0,1,2,3],[0,1,2,3]])
A = sp.csc_matrix(([-1.,-1.,1.,1.], ij), (4,4))

A.data.data

<read-write buffer for 0x108c4d670, size 32, offset 0 at 0x1091994b0>

In [18]:
%%cython

from cpython.mem cimport PyMem_Malloc, PyMem_Free

ctypedef struct mything:
    double * a
    double * b
    
cdef double * avg(mything x, int n):
    z = <double *>PyMem_Malloc(sizeof(double)*n)
    
    cdef int i
    for i in range(n):
        z[i] = (x.a[i] + x.b[i])/2.0
    return z
    # try this. treat it like a c library
    # who conrols the memory returned?
    # time some things. whats the fastest with a large numpy array?
    # is the x.a dereferencing slow? or does cython take care of it?
    # whats going on with z? is it pass by value or pass by reference?


cdef mything convert(double[:] arr):
    cdef mything x = mything(&arr[0], &arr[0])
    return x

def wrapper(arr):
    x = convert(arr)
    print x.a[0]

In [13]:
import numpy as np
arr = np.random.rand(4)

In [14]:
arr

array([ 0.19410064,  0.36458607,  0.41343647,  0.64890736])

In [15]:
wrapper(arr)

0.194100635771


# types
- cython types, floats, doubles mismatch? do we do type-checking/dispatch at the interface level?
- int vs float?

# scope
- how does scope work in cython
- things formed on the stack can be returned in functions?
- those objects live on?
- how do they live on if they are stack variables?

# conversion
- take in two numpy arrays, enter into struct of two float *s
- return a float * answer, which we wrap to a numpy array
- who cleans up?
- take three numpy arrays, write avg of two into the third.

# library interface
- have a C library (maybe just a pure cython function)
- function takes in two float pointers and an int and returns their average
- make versions that take in double and long just to play with
- maybe take in a third pointer for the answer
- if the pointer is NULL, have it allocate space and return the pointer
- then the Cython writer has to decide who is responsible for the memory in the NULL and non NULL case